### 1.SZE different format
#1.
#SZ version A (2016): snapshot(20 cols) + snapshottdw(50 cols)
#without AM PM, 分卷 (SZL2_SNAPSHOT_***.7z.001)
#AM PM txt (SZL2_SNAPSHOT_***_AM.txt)
#AM PM .7z (SZL2_SNAPSHOT_***_AM.7z)
#2.
#SZ version B (2016-2020): hq_snap_spot(35/32 cols) + snap_level_spot(55 cols)
#am&pm，未分卷 (am_hq_snap_spot.7z)
#am&pm，分卷 (am_snap_level_spot.7z.001)
#3.
#深交所数据 (2017, 2019): hq_snap(35/32 cols) + snap_level(55 cols)

### 2.SZE data position
#F: SZ\2016\0104-1230
#J：LEVEL2_shenzhen\2016\04\12 -2016\05\06 
#F: SZ\2017\0103-1229
#F: SZ\2018\0102-1228
#E: SZ\2019\0102-1209 (.pkl)
#H: 2020 data\SZ\0102-0508

### 3.Change columns format

In [1]:
# 2016 version
import pandas as pd
import random
import numpy as np
import glob
import os
from unrar import rarfile
import py7zr
import pickle
import datetime

columns1 = ["Date","OrigTime","SendTime","ercvtime","dbtime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID","PreClosePx",
                   "PxChnage1","PXChange2","openPrice","HighPx","LowPx","close","NumTrades","cum_volume","cum_amount","PE1","PE2","TradingPhase",
                   "totalofferqty", "vwapAsk", "totalbidqty", "vwapBid", "PreNAV", "RealTimeNAV", "WarrantPremiumRate", "UpLimitPx",
                   "DownLimitPx", "TotalLongPosition", "unknown1", "unknown2", "unknown3"]
columns2 = ['Date',"OrigTime","SendTime","ercvtime","dbtime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID",'ask1p','bid1p',
                   "ask1q","bid1q", 'ask2p','bid2p',"ask2q","bid2q",'ask3p','bid3p',"ask3q","bid3q",'ask4p','bid4p',"ask4q","bid4q",'ask5p',
                    'bid5p',"ask5q","bid5q",'ask6p','bid6p',"ask6q","bid6q",'ask7p','bid7p',"ask7q","bid7q",'ask8p','bid8p',"ask8q","bid8q",
                   'ask9p','bid9p',"ask9q","bid9q",'ask10p','bid10p',"ask10q","bid10q","bid1n","NOORDERS_B1","ORDERQTY_B1",
                    "ask1n","NOORDERS_S1","ORDERQTY_S1"]
columns3 = ["Date", "SENDTIME", "SecurityID", "DATATIMESTAMP", "PreClosePx", "openPrice", "HighPx", "LowPx", "close", "NumTrades", "cum_volume",
           "cum_amount", "TOTALLONGPOSITION", "PERATIO1", "PERATIO2", "ENDOFDAYMAKER", "totalofferqty", "vwapAsk", "totalbidqty", "vwapBid"]
columns4 = ["Date", "SENDTIME", "SecurityID", "DATATIMESTAMP", "ask10p", "ask9p", "ask8p", "ask7p", "ask6p", "ask5p", "ask4p", "ask3p", "ask2p",
           "ask1p", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid6p", "bid7p", "bid8p", "bid9p", "bid10p", "ask10q", "ask9q", "ask8q", "ask7q",
           "ask6q", "ask5q", "ask4q", "ask3q", "ask2q", "ask1q", "bid1q", "bid2q", "bid3q", "bid4q", "bid5q", "bid6q", "bid7q", "bid8q", "bid9q", 
           "bid10q", "bid1n", "NOORDERS _B1", "ORDERQTY_B1", "ask1n", "NOORDERS _S1", "ORDERQTY_S1"]
columns5 =  ["Date","OrigTime","SendTime","ercvtime","dbtime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID","PreClosePx",
                   "PxChnage1","PXChange2","openPrice","HighPx","LowPx","close","NumTrades","cum_volume","cum_amount","PE1","PE2","TradingPhase",
                   "totalofferqty", "vwapAsk", "totalbidqty", "vwapBid", "PreNAV", "RealTimeNAV", "WarrantPremiumRate", "UpLimitPx",
                   "DownLimitPx", "TotalLongPosition"]


year = "2016"
df = []
bad = []
readPath = 'J:\\LEVEL2_shenzhen\\' + year + '\\***\\***'
dataPathLs = np.array(glob.glob(readPath))
for data in dataPathLs[2:5]:
    if len(np.array(glob.glob(data +'\\***'))) == 0:
        continue
    
    if data[-5:-3] + data[-2:] == '0413':
        bad.append(data)
        continue
    
    print('---------------------------------------------------------------------------------------------')
    print(data)
    # SZ version A without AM PM, 未分卷
    if len(np.array(glob.glob(data +'\\SZL2_SNAPSHOTDW_***.7z'))) == 1:
        path = r'F:\SZ\2016' 
        d_ate = os.path.basename(glob.glob(data +'\\SZL2_SNAPSHOTDW_***.7z')[0]).split('.')[0].split('_')[2]
        os.chdir(data)
        n_ame = os.path.basename(glob.glob(data +'\\SZL2_SNAPSHOTDW_***.7z')[0]).split('.')[0]
        try:
            a = py7zr.SevenZipFile(data + "\\" + n_ame + '.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + "\\" + n_ame + '.7z')
            bad.append(data + "\\" + n_ame + '.7z')
            continue
        path1 = path + '\\' + d_ate
        a.extractall(path = path1)
        a.close()
        
        snapshot1 = pd.DataFrame()
        snapshot2 = pd.DataFrame()
        if len(np.array(glob.glob(path1 +'\\SZL2_SNAPSHOTDW_***'))) != 2:
            print("Less data!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            bad.append(data + '\\SZL2_SNAPSHOTDW_***')
            print(np.array(glob.glob(path1 +'\\***'))[0])
            continue
        if len(np.array(glob.glob(path1 +'\\SZL2_SNAPSHOT_***'))) != 2:
            print("Less data!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            bad.append(data + '\\SZL2_SNAPSHOT_***')
            print(np.array(glob.glob(path1 +'\\***'))[0])
            continue
        for pp in np.array(glob.glob(path1 +'\\SZL2_SNAPSHOT_***')):
            s1 = pd.read_table(pp, header=None)
            assert(s1.shape[1] == len(columns3))
            s1.columns = columns3
            snapshot1 = pd.concat([snapshot1, s1])
        del s1
        for pp in np.array(glob.glob(path1 +'\\SZL2_SNAPSHOTDW_***')):
            s2 = pd.read_table(pp, header=None)
            assert(s2.shape[1] == len(columns4))
            s2.columns = columns4
            snapshot2 = pd.concat([snapshot2, s2])
        del s2
        snapshot1 = snapshot1[(snapshot1["SecurityID"] < 4000) | (snapshot1["SecurityID"] > 300000)]
        snapshot2 = snapshot2[(snapshot2["SecurityID"] < 4000) | (snapshot2["SecurityID"] > 300000)]
        snapshot1 = snapshot1[snapshot1["DATATIMESTAMP"] != 0]
        snapshot2 = snapshot2[snapshot2["DATATIMESTAMP"] != 0]
        snapshot = pd.merge(snapshot1, snapshot2, on=["Date", "SENDTIME", "SecurityID", "DATATIMESTAMP"], how="outer")
        try:
            assert((snapshot1.shape[0] == snapshot.shape[0]) & (snapshot2.shape[0] == snapshot.shape[0]))
        except:
            if snapshot1.shape[0] == snapshot.shape[0]:
                print("snapshot have more ticks than snapshotdw")
                tt = snapshot[(snapshot["ask1p"].isnull()) & (snapshot["DATATIMESTAMP"] < 145600000)]["DATATIMESTAMP"].unique()
                print(tt)
                print(snapshot[(snapshot["ask1p"].isnull()) & (snapshot["DATATIMESTAMP"].isin(tt))].groupby("DATATIMESTAMP")["ask1p"].size())
                print(np.sort(snapshot[(snapshot["ask1p"].isnull()) & (snapshot["DATATIMESTAMP"] >= 145600000)]["DATATIMESTAMP"].unique()))
                print(snapshot[(snapshot["ask1p"].isnull()) & (snapshot["DATATIMESTAMP"] >= 145600000)].shape[0])
                print((snapshot.shape[0] - snapshot2.shape[0])/snapshot.shape[0])
            elif snapshot2.shape[0] == snapshot.shape[0]:
                print("snapshotdw have more ticks than snapshot")
                print(snapshot[snapshot["close"].isnull()]["DATATIMESTAMP"].unique().min())
                print((snapshot.shape[0] - snapshot1.shape[0])/snapshot.shape[0])
            else:
                print("snapshotdw don't join with snapshot")
            snapshot = pd.merge(snapshot1, snapshot2, on=["Date", "SENDTIME", "SecurityID", "DATATIMESTAMP"])
        del snapshot1
        del snapshot2
        snapshot['time'] = snapshot["DATATIMESTAMP"]
        snapshot["SecurityID"] = snapshot["SecurityID"].astype(int)
        snapshot = snapshot.rename(columns={"SecurityID":"StockID"})
        snapshot["OrigTime"] = snapshot["Date"] * 1000000000 + snapshot["DATATIMESTAMP"]
        snapshot.loc[snapshot["openPrice"] == 0, "LowPx"] = 0
        
        print("SZ version A without AM PM，未分卷")
        
    # SZ version A without AM PM, 分卷
    elif len(np.array(glob.glob(data +'\\SZL2_SNAPSHOTDW_***.7z.001'))) == 1:
        path = r'F:\SZ\2016' 
        d_ate = os.path.basename(glob.glob(data +'\\SZL2_SNAPSHOTDW_***.7z.001')[0]).split('.')[0].split('_')[2]
        os.chdir(data)
        n_ame = os.path.basename(glob.glob(data +'\\SZL2_SNAPSHOTDW_***.7z.001')[0]).split('.')[0]
        os.system("copy /b " + n_ame + ".7z.* " + n_ame + ".7z")
        try:
            a = py7zr.SevenZipFile(data + "\\" + n_ame + '.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + "\\" + n_ame + '.7z')
            bad.append(data + "\\" + n_ame + '.7z')
            continue
        path1 = path + '\\' + d_ate
        a.extractall(path = path1)
        a.close()
        
        snapshot1 = pd.DataFrame()
        snapshot2 = pd.DataFrame()
        if len(np.array(glob.glob(path1 +'\\SZL2_SNAPSHOTDW_***'))) != 2:
            print("Less data!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            bad.append(data + '\\SZL2_SNAPSHOTDW_***')
            print(np.array(glob.glob(path1 +'\\***'))[0])
            continue
        elif len(np.array(glob.glob(path1 +'\\SZL2_SNAPSHOT_***'))) != 2:
            print("Less data!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            bad.append(data + '\\SZL2_SNAPSHOT_***')
            print(np.array(glob.glob(path1 +'\\***'))[0])
            continue
        for pp in np.array(glob.glob(path1 +'\\SZL2_SNAPSHOT_***')):
            s1 = pd.read_table(pp, header=None)
            assert(s1.shape[1] == len(columns3))
            s1.columns = columns3
            snapshot1 = pd.concat([snapshot1, s1])
        del s1
        for pp in np.array(glob.glob(path1 +'\\SZL2_SNAPSHOTDW_***')):
            s2 = pd.read_table(pp, header=None)
            assert(s2.shape[1] == len(columns4))
            s2.columns = columns4
            snapshot2 = pd.concat([snapshot2, s2])
        del s2
        snapshot1 = snapshot1[(snapshot1["SecurityID"] < 4000) | (snapshot1["SecurityID"] > 300000)]
        snapshot2 = snapshot2[(snapshot2["SecurityID"] < 4000) | (snapshot2["SecurityID"] > 300000)]
        snapshot1 = snapshot1[snapshot1["DATATIMESTAMP"] != 0]
        snapshot2 = snapshot2[snapshot2["DATATIMESTAMP"] != 0]
        snapshot = pd.merge(snapshot1, snapshot2, on=["Date", "SENDTIME", "SecurityID", "DATATIMESTAMP"], how="outer")
        try:
            assert((snapshot1.shape[0] == snapshot.shape[0]) & (snapshot2.shape[0] == snapshot.shape[0]))
        except:
            if snapshot1.shape[0] == snapshot.shape[0]:
                print("snapshot have more ticks than snapshotdw")
                print(snapshot[(snapshot["ask1p"].isnull()) & (snapshot["DATATIMESTAMP"] < 145600000)]["DATATIMESTAMP"].unique())
                print(snapshot[(snapshot["ask1p"].isnull()) & (snapshot["DATATIMESTAMP"] < 145600000)].shape[0])
                print(np.sort(snapshot[(snapshot["ask1p"].isnull()) & (snapshot["DATATIMESTAMP"] >= 145600000)]["DATATIMESTAMP"].unique()))
                print(snapshot[(snapshot["ask1p"].isnull()) & (snapshot["DATATIMESTAMP"] >= 145600000)].shape[0])
                print((snapshot.shape[0] - snapshot2.shape[0])/snapshot.shape[0])
            elif snapshot2.shape[0] == snapshot.shape[0]:
                print("snapshotdw have more ticks than snapshot")
                print(snapshot[snapshot["close"].isnull()]["DATATIMESTAMP"].unique().min())
                print((snapshot.shape[0] - snapshot1.shape[0])/snapshot.shape[0])
            else:
                print("snapshotdw don't join with snapshot")
            snapshot = pd.merge(snapshot1, snapshot2, on=["Date", "SENDTIME", "SecurityID", "DATATIMESTAMP"])
        del snapshot1
        del snapshot2
        snapshot['time'] = snapshot["DATATIMESTAMP"]
        snapshot["SecurityID"] = snapshot["SecurityID"].astype(int)
        snapshot = snapshot.rename(columns={"SecurityID":"StockID"})
        snapshot["OrigTime"] = snapshot["Date"] * 1000000000 + snapshot["DATATIMESTAMP"]
        snapshot.loc[snapshot["openPrice"] == 0, "LowPx"] = 0
        
        print("SZ version A without AM PM，分卷")
        
    # SZ version A AM PM
    # I:\\LEVEL2_shenzhen\\2016\\04\\19\\SZL2_SNAPSHOT_20160419_AM.7z 58
    # I:\\LEVEL2_shenzhen\\2016\\05\\06\\SZL2_SNAPSHOT_20160506_AM.txt 59
    elif len(np.array(glob.glob(data +'\\SZL2_SNAPSHOTDW_***_AM.txt'))) == 1:
        snap1 = pd.read_table(np.array(glob.glob(data +'\\SZL2_SNAPSHOTDW_***_AM.txt'))[0], header=None)
        assert(snap1.shape[1] == len(columns4))
        snap1.columns = columns4
        snap2 = pd.read_table(np.array(glob.glob(data +'\\SZL2_SNAPSHOTDW_***_PM.txt'))[0], header=None)
        assert(snap2.shape[1] == len(columns4))
        snap2.columns = columns4
        snapshot2 = pd.concat([snap1, snap2]).sort_values(by=["SecurityID", "DATATIMESTAMP"])
        del snap1
        del snap2
        
        snap1 = pd.read_table(np.array(glob.glob(data +'\\SZL2_SNAPSHOT_***_AM.txt'))[0], header=None)
        assert(snap1.shape[1] == len(columns3))
        snap1.columns = columns3
        snap2 = pd.read_table(np.array(glob.glob(data +'\\SZL2_SNAPSHOT_***_PM.txt'))[0], header=None)
        assert(snap2.shape[1] == len(columns3))
        snap2.columns = columns3
        snapshot1 = pd.concat([snap1, snap2]).sort_values(by=["SecurityID", "DATATIMESTAMP"])
        del snap1
        del snap2
        
        snapshot1 = snapshot1[(snapshot1["SecurityID"] < 4000) | (snapshot1["SecurityID"] > 300000)]
        snapshot2 = snapshot2[(snapshot2["SecurityID"] < 4000) | (snapshot2["SecurityID"] > 300000)]
        snapshot1 = snapshot1[snapshot1["DATATIMESTAMP"] != 0]
        snapshot2 = snapshot2[snapshot2["DATATIMESTAMP"] != 0]
        snapshot = pd.merge(snapshot1, snapshot2, on=["Date", "SENDTIME", "SecurityID", "DATATIMESTAMP"], how="outer")
        
        try:
            assert((snapshot1.shape[0] == snapshot.shape[0]) & (snapshot2.shape[0] == snapshot.shape[0]))
        except:
            if snapshot1.shape[0] == snapshot.shape[0]:
                print("snapshot have more ticks than snapshotdw")
                print(snapshot[(snapshot["ask1p"].isnull()) & (snapshot["DATATIMESTAMP"] < 145600000)]["DATATIMESTAMP"].unique())
                print(snapshot[(snapshot["ask1p"].isnull()) & (snapshot["DATATIMESTAMP"] < 145600000)].shape[0])
                print(np.sort(snapshot[(snapshot["ask1p"].isnull()) & (snapshot["DATATIMESTAMP"] >= 145600000)]["DATATIMESTAMP"].unique()))
                print(snapshot[(snapshot["ask1p"].isnull()) & (snapshot["DATATIMESTAMP"] >= 145600000)].shape[0])
                print((snapshot.shape[0] - snapshot2.shape[0])/snapshot.shape[0])
            elif snapshot2.shape[0] == snapshot.shape[0]:
                print("snapshotdw have more ticks than snapshot")
                print(snapshot[snapshot["close"].isnull()]["DATATIMESTAMP"].unique().min())
                print((snapshot.shape[0] - snapshot1.shape[0])/snapshot.shape[0])
            else:
                print("snapshotdw don't join with snapshot")
            snapshot = pd.merge(snapshot1, snapshot2, on=["Date", "SENDTIME", "SecurityID", "DATATIMESTAMP"])
            
        del snapshot1
        del snapshot2
        snapshot["SecurityID"] = snapshot["SecurityID"].astype(int)
        snapshot = snapshot.rename(columns={"SecurityID":"StockID"})
        snapshot['time'] = snapshot["DATATIMESTAMP"]
        snapshot["OrigTime"] = snapshot["Date"] * 1000000000 + snapshot["DATATIMESTAMP"]
        snapshot.loc[snapshot["openPrice"] == 0, "LowPx"] = 0
        
        print("SZ version A AM PM, txt")
                
        
    elif len(np.array(glob.glob(data +'\\SZL2_SNAPSHOTDW_***_AM.7z'))) == 1:
        path = r'F:\SZ\2016' 
        d_ate = os.path.basename(glob.glob(data +'\\SZL2_SNAPSHOTDW_***_AM.7z')[0]).split('.')[0].split('_')[2]
        os.chdir(data)
        n_ame = os.path.basename(glob.glob(data +'\\SZL2_SNAPSHOTDW_***_AM.7z')[0]).split('.')[0]
        try:
            a = py7zr.SevenZipFile(data + "\\" + n_ame + '.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + "\\" + n_ame + '.7z')
            bad.append(data + "\\" + n_ame + '.7z')
            continue
        path1 = path + '\\' + d_ate
        a.extractall(path = path1)
        
        n_ame1 = os.path.basename(glob.glob(data +'\\SZL2_SNAPSHOTDW_***_PM.7z')[0]).split('.')[0]
        try:
            a = py7zr.SevenZipFile(data + "\\" + n_ame1 + '.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + "\\" + n_ame1 + '.7z')
            bad.append(data + "\\" + n_ame1 + '.7z')
            continue
        a.extractall(path = path1)
        
        if len(np.array(glob.glob(path1 +'\\SZL2_SNAPSHOT_***'))) != 2:
            print("Less data!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            bad.append(data + '\\SZL2_SNAPSHOT_***')
            print(np.array(glob.glob(path1 +'\\***'))[0])
            continue
        snap1 = pd.read_table(np.array(glob.glob(path1 +'\\SZL2_SNAPSHOT_***_AM.txt'))[0], header=None)
        assert(snap1.shape[1] == len(columns3))
        snap1.columns = columns3
        snap2 = pd.read_table(np.array(glob.glob(path1 +'\\SZL2_SNAPSHOT_***_PM.txt'))[0], header=None)
        assert(snap2.shape[1] == len(columns3))
        snap2.columns = columns3
        snapshot1 = pd.concat([snap1, snap2]).sort_values(by=["SecurityID", "DATATIMESTAMP"])
        del snap1
        del snap2
        
        snap1 = pd.read_table(np.array(glob.glob(path1 +'\\SZL2_SNAPSHOTDW_***_AM.txt'))[0], header=None)
        assert(snap1.shape[1] == len(columns4))
        snap1.columns = columns4
        snap2 = pd.read_table(np.array(glob.glob(path1 +'\\SZL2_SNAPSHOTDW_***_PM.txt'))[0], header=None)
        assert(snap2.shape[1] == len(columns4))
        snap2.columns = columns4
        snapshot2 = pd.concat([snap1, snap2]).sort_values(by=["SecurityID", "DATATIMESTAMP"])
        del snap1
        del snap2
        
        snapshot1 = snapshot1[(snapshot1["SecurityID"] < 4000) | (snapshot1["SecurityID"] > 300000)]
        snapshot2 = snapshot2[(snapshot2["SecurityID"] < 4000) | (snapshot2["SecurityID"] > 300000)]
        snapshot1 = snapshot1[snapshot1["DATATIMESTAMP"] != 0]
        snapshot2 = snapshot2[snapshot2["DATATIMESTAMP"] != 0]
        snapshot = pd.merge(snapshot1, snapshot2, on=["Date", "SENDTIME", "SecurityID", "DATATIMESTAMP"], how="outer")
        
        try:
            assert((snapshot1.shape[0] == snapshot.shape[0]) & (snapshot2.shape[0] == snapshot.shape[0]))
        except:
            if snapshot1.shape[0] == snapshot.shape[0]:
                print("snapshot have more ticks than snapshotdw")
                print(snapshot[(snapshot["ask1p"].isnull()) & (snapshot["DATATIMESTAMP"] < 145600000)]["DATATIMESTAMP"].unique())
                print(snapshot[(snapshot["ask1p"].isnull()) & (snapshot["DATATIMESTAMP"] < 145600000)].shape[0])
                print(np.sort(snapshot[(snapshot["ask1p"].isnull()) & (snapshot["DATATIMESTAMP"] >= 145600000)]["DATATIMESTAMP"].unique()))
                print(snapshot[(snapshot["ask1p"].isnull()) & (snapshot["DATATIMESTAMP"] >= 145600000)].shape[0])
                print((snapshot.shape[0] - snapshot2.shape[0])/snapshot.shape[0])
            elif snapshot2.shape[0] == snapshot.shape[0]:
                print("snapshotdw have more ticks than snapshot")
                print(snapshot[snapshot["close"].isnull()]["DATATIMESTAMP"].unique().min())
                print((snapshot.shape[0] - snapshot1.shape[0])/snapshot.shape[0])
            else:
                print("snapshotdw don't join with snapshot")
            snapshot = pd.merge(snapshot1, snapshot2, on=["Date", "SENDTIME", "SecurityID", "DATATIMESTAMP"])
        del snapshot1
        del snapshot2
        
        snapshot["SecurityID"] = snapshot["SecurityID"].astype(int)
        snapshot = snapshot.rename(columns={"SecurityID":"StockID"})
        snapshot['time'] = snapshot["DATATIMESTAMP"]
        snapshot["OrigTime"] = snapshot["Date"] * 1000000000 + snapshot["DATATIMESTAMP"]
        snapshot.loc[snapshot["openPrice"] == 0, "LowPx"] = 0
        
        print("SZ version A AM PM, .7z")
 
    # am & pm
    # 'I:\\LEVEL2_shenzhen\\2016\\05\\09\\am_snap_level_spot.7z'
    # 'I:\\LEVEL2_shenzhen\\2016\\05\\10\\am_snap_level_spot.7z.001'
    
    elif len(np.array(glob.glob(data +'\\am_snap_level_spot.7z'))) == 1:
        date = glob.glob(data +'\\am_snap_level_spot.7z')[0][19:23] + glob.glob(data +'\\am_snap_level_spot.7z')[0][24:26] + glob.glob(data +'\\am_snap_level_spot.7z')[0][27:29]
        path = r'F:\SZ\2016' 
        os.chdir(data)
        try:
            a = py7zr.SevenZipFile(data + '\\am_snap_level_spot.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + '\\am_snap_level_spot.7z')
            bad.append(data + '\\am_snap_level_spot.7z')
            continue
        path1 = path + '\\' + date
        a.extractall(path = path1)
        a.close()
        try:
            a = py7zr.SevenZipFile(data + '\\pm_snap_level_spot.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + '\\pm_snap_level_spot.7z')
            bad.append(data + '\\pm_snap_level_spot.7z')
            continue
        a.extractall(path = path1)
        a.close()
        
        if len(np.array(glob.glob(path1 +'\\***_hq_snap_spot.txt'))) != 2:
            print("Less data!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            bad.append(data + '\\***_hq_snap_spot.txt')
            print(np.array(glob.glob(path1 +'\\***'))[0])
            continue
        am_snap1 = pd.read_table(path1 + "\\am_hq_snap_spot.txt", header=None)
        try:
            assert(am_snap1.shape[1] == len(columns1))
            am_snap1.columns = columns1       
        except:
            assert(am_snap1.shape[1] == len(columns5))
            am_snap1.columns = columns5     
        pm_snap1 = pd.read_table(path1 + "\\pm_hq_snap_spot.txt", header=None)
        try:
            assert(pm_snap1.shape[1] == len(columns1))
            pm_snap1.columns = columns1       
        except:
            assert(pm_snap1.shape[1] == len(columns5))
            pm_snap1.columns = columns5     
        snapshot1 = pd.concat([am_snap1, pm_snap1]).sort_values(by=["SecurityID", "OrigTime"])
        del am_snap1
        del pm_snap1
        
        am_snap1 = pd.read_table(path1 + "\\am_snap_level_spot.txt", header=None)
        assert(am_snap1.shape[1] == len(columns2))
        am_snap1.columns = columns2       
        pm_snap1 = pd.read_table(path1 + "\\pm_snap_level_spot.txt", header=None)
        assert(pm_snap1.shape[1] == len(columns2))
        pm_snap1.columns = columns2       
        snapshot2 = pd.concat([am_snap1, pm_snap1]).sort_values(by=["SecurityID", "OrigTime"])
        del am_snap1
        del pm_snap1
        
        snapshot1 = snapshot1[(snapshot1["SecurityID"] < 4000) | (snapshot1["SecurityID"] > 300000)]
        snapshot2 = snapshot2[(snapshot2["SecurityID"] < 4000) | (snapshot2["SecurityID"] > 300000)]
        snapshot1['time'] = (snapshot1['OrigTime'] - int(snapshot1['OrigTime'].iloc[0]//1000000000*1000000000)).astype(int)
        snapshot2['time'] = (snapshot2['OrigTime'] - int(snapshot2['OrigTime'].iloc[0]//1000000000*1000000000)).astype(int)
        snapshot = pd.merge(snapshot1, snapshot2, on=['Date',"OrigTime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID", "time"], how="outer")
        try:
            assert((snapshot.shape[0] == snapshot1.shape[0]) & (snapshot.shape[0] == snapshot2.shape[0]))
        except:
            if snapshot.shape[0] == snapshot1.shape[0]:
                print("snapshot1 have more ticks than snapshot2")
                if all(snapshot[(snapshot["dbtime_y"].isnull()) & ((snapshot["SecurityID"] < 4000) | (snapshot["SecurityID"] > 200000))]["time"].unique() > 150000000):
                    print("More ticks happens after 15:00")
                elif all(snapshot[(snapshot["dbtime_y"].isnull()) & ((snapshot["SecurityID"] < 4000) | (snapshot["SecurityID"] > 200000))]["time"].unique() < 93000000):
                    print("More ticks happens before 9:30")
                else:
                    print("There are ticks happens before 15:00, after 9:30")
            elif snapshot.shape[0] == snapshot2.shape[0]:
                print("snapshot2 have more ticks than snapshot1")
                if all(snapshot[(snapshot["dbtime_x"].isnull()) & ((snapshot["SecurityID"] < 4000) | (snapshot["SecurityID"] > 200000))]["time"].unique() > 150000000):
                    print("More ticks happens after 15:00")
                elif all(snapshot[(snapshot["dbtime_x"].isnull()) & ((snapshot["SecurityID"] < 4000) | (snapshot["SecurityID"] > 200000))]["time"].unique() < 93000000):
                    print("More ticks happens before 9:30")
                else:
                    print("There are ticks happens before 15:00, after 9:30")
            else:
                print("snapshot2 don't join with snapshot1")
            snapshot = pd.merge(snapshot1, snapshot2, on=['Date',"OrigTime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID", "time"])
        del snapshot1
        del snapshot2
           
        snapshot = snapshot.rename(columns={"SecurityID":"StockID"})
        print(int(snapshot['OrigTime'].iloc[0]//1000000000*1000000000))
        
        print("am & pm data 未分卷")
        
    elif len(np.array(glob.glob(data +'\\am_snap_level_spot.7z.001'))) == 1:    
        date = glob.glob(data +'\\am_snap_level_spot.7z.001')[0][19:23] + glob.glob(data +'\\am_snap_level_spot.7z.001')[0][24:26] + glob.glob(data +'\\am_snap_level_spot.7z.001')[0][27:29]
        path = r'F:\SZ\2016' 
        os.chdir(data)
        os.system("copy /b am_snap_level_spot.7z.* am_snap_level_spot.7z")
        try:
            a = py7zr.SevenZipFile(data + '\\am_snap_level_spot.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + '\\am_snap_level_spot.7z')
            bad.append(data + '\\am_snap_level_spot.7z')
            continue
        path1 = path + '\\' + date
        a.extractall(path = path1)
        a.close()
        
        os.system("copy /b pm_snap_level_spot.7z.* pm_snap_level_spot.7z")
        try:
            a = py7zr.SevenZipFile(data + '\\pm_snap_level_spot.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + '\\pm_snap_level_spot.7z')
            bad.append(data + '\\pm_snap_level_spot.7z')
            continue
        a.extractall(path = path1)
        a.close()
        
        if len(np.array(glob.glob(path1 +'\\***_hq_snap_spot.txt'))) != 2:
            print("Less data!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            bad.append(data + '\\***_hq_snap_spot.txt')
            print(np.array(glob.glob(path1 +'\\***'))[0])
            continue
        am_snap1 = pd.read_table(path1 + "\\am_hq_snap_spot.txt", header=None)
        try:
            assert(am_snap1.shape[1] == len(columns1))
            am_snap1.columns = columns1       
        except:
            assert(am_snap1.shape[1] == len(columns5))
            am_snap1.columns = columns5       
        pm_snap1 = pd.read_table(path1 + "\\pm_hq_snap_spot.txt", header=None)
        try:
            assert(pm_snap1.shape[1] == len(columns1))
            pm_snap1.columns = columns1       
        except:
            assert(pm_snap1.shape[1] == len(columns5))
            pm_snap1.columns = columns5     
        snapshot1 = pd.concat([am_snap1, pm_snap1]).sort_values(by=["SecurityID", "OrigTime"])
        del am_snap1
        del pm_snap1
        
        am_snap1 = pd.read_table(path1 + "\\am_snap_level_spot.txt", header=None)
        assert(am_snap1.shape[1] == len(columns2))
        am_snap1.columns = columns2        
        pm_snap1 = pd.read_table(path1 + "\\pm_snap_level_spot.txt", header=None)
        assert(pm_snap1.shape[1] == len(columns2))
        pm_snap1.columns = columns2       
        snapshot2 = pd.concat([am_snap1, pm_snap1]).sort_values(by=["SecurityID", "OrigTime"])
        del am_snap1
        del pm_snap1
        
        snapshot1 = snapshot1[(snapshot1["SecurityID"] < 4000) | (snapshot1["SecurityID"] > 300000)]
        snapshot2 = snapshot2[(snapshot2["SecurityID"] < 4000) | (snapshot2["SecurityID"] > 300000)]
        snapshot1['time'] = (snapshot1['OrigTime'] - int(snapshot1['OrigTime'].iloc[0]//1000000000*1000000000)).astype(int)
        snapshot2['time'] = (snapshot2['OrigTime'] - int(snapshot2['OrigTime'].iloc[0]//1000000000*1000000000)).astype(int)
        snapshot = pd.merge(snapshot1, snapshot2, on=['Date',"OrigTime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID", "time"], how="outer")
        try:
            assert((snapshot.shape[0] == snapshot1.shape[0]) & (snapshot.shape[0] == snapshot2.shape[0]))
        except:
            if snapshot.shape[0] == snapshot1.shape[0]:
                print("snapshot1 have more ticks than snapshot2")
                if all(snapshot[(snapshot["dbtime_y"].isnull()) & ((snapshot["SecurityID"] < 4000) | (snapshot["SecurityID"] > 200000))]["time"].unique() > 150000000):
                    print("More ticks happens after 15:00")
                elif all(snapshot[(snapshot["dbtime_y"].isnull()) & ((snapshot["SecurityID"] < 4000) | (snapshot["SecurityID"] > 200000))]["time"].unique() < 93000000):
                    print("More ticks happens before 9:30")
                else:
                    print("There are ticks happens before 15:00, after 9:30")
            elif snapshot.shape[0] == snapshot2.shape[0]:
                print("snapshot2 have more ticks than snapshot1")
                if all(snapshot[(snapshot["dbtime_x"].isnull()) & ((snapshot["SecurityID"] < 4000) | (snapshot["SecurityID"] > 200000))]["time"].unique() > 150000000):
                    print("More ticks happens after 15:00")
                elif all(snapshot[(snapshot["dbtime_x"].isnull()) & ((snapshot["SecurityID"] < 4000) | (snapshot["SecurityID"] > 200000))]["time"].unique() < 93000000):
                    print("More ticks happens before 9:30")
                else:
                    print("There are ticks happens before 15:00, after 9:30")
            else:
                print("snapshot2 don't join with snapshot1")
            snapshot = pd.merge(snapshot1, snapshot2, on=['Date',"OrigTime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID", "time"])
        del snapshot1
        del snapshot2
        
        snapshot = snapshot.rename(columns={"SecurityID":"StockID"})
        print(int(snapshot['OrigTime'].iloc[0]//1000000000*1000000000))
        
        print("am & pm data 分卷") 
        
        
    snapshot["clockAtArrival"] = snapshot["OrigTime"].astype(str).apply(lambda x: np.int64(datetime.datetime.strptime(x, '%Y%m%d%H%M%S%f').timestamp()*1e6))
    snapshot = snapshot.rename(columns={"Date":"date", "NumTrades":"cum_tradesCnt", "HighPx":"high", "LowPx":"low", "totalofferqty":
                                   "totalAskQuantity", "totalbidqty":"totalBidQuantity", "PreClosePx":"prevClose", "openPrice":"open"})
    snapshot["ID"] = snapshot["StockID"] + 2000000
    snapshot["ordering"] = snapshot.groupby("ID").cumcount()
    snapshot["ordering"] = snapshot["ordering"] + 1
    snapshot["time"] = snapshot["time"].astype('int64') * 1000
    

    snapshot.loc[(~snapshot["ORDERQTY_B1"].isnull()), "ORDERQTY_B1"]=snapshot[(~snapshot["ORDERQTY_B1"].isnull())]["ORDERQTY_B1"].apply(lambda x: [int(i) for i in x.split('|')])
    snapshot.loc[(~snapshot["ORDERQTY_S1"].isnull()), "ORDERQTY_S1"]=snapshot[(~snapshot["ORDERQTY_S1"].isnull())]["ORDERQTY_S1"].apply(lambda x: [int(i) for i in x.split('|')])
    ## lambda x: [int(i) for i in x.split('|')]
    for i in range(1, 51):
        snapshot["bid1Top" + str(i) + 'q'] = 0
        snapshot["ask1Top" + str(i) + 'q'] = 0
    for i in range(1, 51):
        snapshot.loc[i <= snapshot["bid1n"], "bid1Top" + str(i) + 'q'] = snapshot.loc[i <= snapshot["bid1n"], "ORDERQTY_B1"].apply(lambda x: x[i-1])
        snapshot.loc[i <= snapshot["ask1n"], "ask1Top" + str(i) + 'q'] = snapshot.loc[i <= snapshot["ask1n"], "ORDERQTY_S1"].apply(lambda x: x[i-1])    

        
        
    for columns in ["cum_tradesCnt", "cum_volume", 'ask1q','ask2q','ask3q','ask4q','ask5q','ask6q','ask7q','ask8q','ask9q',
                        'ask10q', 'bid1q','bid2q','bid3q','bid4q','bid5q','bid6q','bid7q','bid8q','bid9q','bid10q',
                    "totalBidQuantity", "totalAskQuantity", "bid1n", "ask1n"]:
        snapshot[columns] = snapshot[columns].astype('int64')
    
    
    for cols in ["prevClose", 'open', "high", "low", "close", 'bid10p','bid9p','bid8p','bid7p','bid6p','bid5p','bid4p','bid3p',
             'bid2p','bid1p','ask1p','ask2p','ask3p','ask4p','ask5p','ask6p','ask7p','ask8p','ask9p','ask10p', 'vwapBid', "vwapAsk"]:
        snapshot[cols] = (snapshot[cols] * 10000).round(0).astype('int64')
    
    for cols in ["cum_amount"]:
        snapshot[cols] = (snapshot[cols] * 10000).round(0).astype('int64')
    
    snapshot['datetime'] = snapshot["clockAtArrival"].apply(lambda x: datetime.datetime.fromtimestamp(x/1e6))
    
    for cols in ['bid10n', 'bid9n', 'bid8n', 'bid7n', 'bid6n', 'bid5n', 'bid4n', 'bid3n', 'bid2n', 'ask2n', 'ask3n', 'ask4n', 'ask5n', 'ask6n','ask7n', 
                 'ask8n', 'ask9n', 'ask10n', "totalBidOrders",'totalAskOrders','totalBidLevels', 'totalAskLevels', 'bidTradeMaxDuration', 'askTradeMaxDuration', 'cum_canceledBuyOrders', 'cum_canceledBuyVolume',
        "cum_canceledBuyAmount", "cum_canceledSellOrders", 'cum_canceledSellVolume',"cum_canceledSellAmount"]:
        snapshot[cols] = 0
    
    
    snapshot = snapshot[["date", "time", "clockAtArrival", "datetime", "ID", "ordering", "cum_tradesCnt", "cum_volume", "cum_amount", "prevClose",
                            "open", "high", "low", "close", 'bid10p','bid9p','bid8p','bid7p','bid6p','bid5p','bid4p','bid3p','bid2p','bid1p',
                            'ask1p','ask2p','ask3p','ask4p','ask5p','ask6p','ask7p','ask8p','ask9p','ask10p', 'bid10q','bid9q','bid8q',
                             'bid7q','bid6q','bid5q','bid4q','bid3q','bid2q','bid1q', 'ask1q','ask2q','ask3q','ask4q','ask5q','ask6q',
                             'ask7q','ask8q','ask9q','ask10q', 'bid10n', 'bid9n', 'bid8n', 'bid7n', 'bid6n', 'bid5n', 'bid4n', 'bid3n', 'bid2n', 'bid1n', 
                             'ask1n', 'ask2n', 'ask3n', 'ask4n', 'ask5n', 'ask6n','ask7n', 'ask8n', 'ask9n', 'ask10n','bid1Top1q','bid1Top2q','bid1Top3q','bid1Top4q','bid1Top5q','bid1Top6q',
        'bid1Top7q','bid1Top8q','bid1Top9q','bid1Top10q','bid1Top11q','bid1Top12q','bid1Top13q','bid1Top14q','bid1Top15q','bid1Top16q','bid1Top17q','bid1Top18q',
        'bid1Top19q','bid1Top20q','bid1Top21q','bid1Top22q','bid1Top23q','bid1Top24q','bid1Top25q','bid1Top26q','bid1Top27q','bid1Top28q','bid1Top29q',
        'bid1Top30q','bid1Top31q','bid1Top32q','bid1Top33q','bid1Top34q','bid1Top35q','bid1Top36q','bid1Top37q','bid1Top38q','bid1Top39q','bid1Top40q',
        'bid1Top41q','bid1Top42q','bid1Top43q','bid1Top44q','bid1Top45q','bid1Top46q','bid1Top47q','bid1Top48q','bid1Top49q','bid1Top50q', 'ask1Top1q',
        'ask1Top2q','ask1Top3q','ask1Top4q','ask1Top5q','ask1Top6q','ask1Top7q','ask1Top8q','ask1Top9q','ask1Top10q','ask1Top11q','ask1Top12q','ask1Top13q',
        'ask1Top14q','ask1Top15q','ask1Top16q','ask1Top17q','ask1Top18q','ask1Top19q','ask1Top20q','ask1Top21q','ask1Top22q','ask1Top23q',
        'ask1Top24q','ask1Top25q','ask1Top26q','ask1Top27q','ask1Top28q','ask1Top29q','ask1Top30q','ask1Top31q','ask1Top32q','ask1Top33q',
        'ask1Top34q','ask1Top35q','ask1Top36q','ask1Top37q','ask1Top38q','ask1Top39q','ask1Top40q','ask1Top41q','ask1Top42q','ask1Top43q',
        'ask1Top44q','ask1Top45q','ask1Top46q','ask1Top47q','ask1Top48q','ask1Top49q','ask1Top50q',"totalBidQuantity", "totalAskQuantity","vwapBid", "vwapAsk",
        "totalBidOrders",'totalAskOrders','totalBidLevels', 'totalAskLevels', 'bidTradeMaxDuration', 'askTradeMaxDuration', 'cum_canceledBuyOrders', 'cum_canceledBuyVolume',
        "cum_canceledBuyAmount", "cum_canceledSellOrders", 'cum_canceledSellVolume',"cum_canceledSellAmount"]]
print(bad)    

---------------------------------------------------------------------------------------------
J:\LEVEL2_shenzhen\2016\01\06


C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (46,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


snapshot have more ticks than snapshotdw
[93001460]
618
[145641820 145642720 145646140 ... 145959870 145959890 145959980]
6606
0.0009022897851528616
SZ version A without AM PM，分卷
---------------------------------------------------------------------------------------------
J:\LEVEL2_shenzhen\2016\01\07
Less data!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
F:\SZ\2016\20160107\SZL2_SNAPSHOTDW_20160107.7z.001
---------------------------------------------------------------------------------------------
J:\LEVEL2_shenzhen\2016\01\08
snapshot have more ticks than snapshotdw
[91600160]
5118
[145646800 145650000 145651980 ... 145959440 145959860 145959930]
6253
0.001426953317255481
SZ version A without AM PM，分卷
['J:\\LEVEL2_shenzhen\\2016\\01\\07\\SZL2_SNAPSHOTDW_***']


In [8]:
snapshot1[(snapshot1["SecurityID"] == 1)].head(20)

,Date,SENDTIME,SecurityID,DATATIMESTAMP,PreClosePx,openPrice,HighPx,LowPx,close,NumTrades,cum_volume,cum_amount,TOTALLONGPOSITION,PERATIO1,PERATIO2,ENDOFDAYMAKER,totalofferqty,vwapAsk,totalbidqty,vwapBid
0,20160105,92501,1,92459350,11.33,0.00,0.00,999999.999,0.00,0,0,0.0,0,8.21,0.0,N,0,0.00,0,0.00
6373,20160105,92506,1,92500810,11.33,11.27,11.27,11.270,11.27,438,1308400,14745668.0,0,8.21,0.0,N,4256019,11.85,5397500,10.76
8257,20160105,92508,1,92500810,11.33,11.27,11.27,11.270,11.27,438,1308400,14745668.0,0,8.21,0.0,N,4256019,11.85,5397500,10.76
12326,20160105,92511,1,92500810,11.33,11.27,11.27,11.270,11.27,438,1308400,14745668.0,0,8.21,0.0,N,4256019,11.85,5397500,10.76
16395,20160105,92514,1,92500810,11.33,11.27,11.27,11.270,11.27,438,1308400,14745668.0,0,8.21,0.0,N,4256019,11.85,5397500,10.76
20464,20160105,92517,1,92500810,11.33,11.27,11.27,11.270,11.27,438,1308400,14745668.0,0,8.21,0.0,N,4256019,11.85,5397500,10.76
24633,20160105,92520,1,92500810,11.33,11.27,11.27,11.270,11.27,438,1308400,14745668.0,0,8.21,0.0,N,4256019,11.85,5397500,10.76
28702,20160105,92523,1,92500810,11.33,11.27,11.27,11.270,11.27,438,1308400,14745668.0,0,8.21,0.0,N,4256019,11.85,5397500,10.76
32871,20160105,92526,1,92500810,11.33,11.27,11.27,11.270,11.27,438,1308400,14745668.0,0,8.21,0.0,N,4256019,11.85,5397500,10.76
38925,20160105,92529,1,92500810,11.33,11.27,11.27,11.270,11.27,438,1308400,14745668.0,0,8.21,0.0,N,4256019,11.85,5397500,10.76


In [9]:
snapshot2[(snapshot2["SecurityID"] == 1)].head(20)

,Date,SENDTIME,SecurityID,DATATIMESTAMP,ask10p,ask9p,ask8p,ask7p,ask6p,ask5p,ask4p,ask3p,ask2p,ask1p,bid1p,bid2p,bid3p,bid4p,bid5p,bid6p,bid7p,bid8p,bid9p,bid10p,ask10q,ask9q,ask8q,ask7q,ask6q,ask5q,ask4q,ask3q,ask2q,ask1q,bid1q,bid2q,bid3q,bid4q,bid5q,bid6q,bid7q,bid8q,bid9q,bid10q,bid1n,NOORDERS _B1,ORDERQTY_B1,ask1n,NOORDERS _S1,ORDERQTY_S1
0,20160105,92501,1,92459350,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,11.28,11.28,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,1273600,63770,0,0,0,0,0,0,0,0,1273600,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,NaN
2546,20160105,92506,1,92500810,11.37,11.36,11.35,11.33,11.32,11.31,11.3,11.29,11.28,11.27,11.26,11.25,11.24,11.23,11.22,11.21,11.2,11.19,11.18,11.17,89731,157200,386700,194100,41700,86100,475921,8100,500,2000,134200,24600,800,17400,52100,55500,66000,13700,14600,400,12,12,3500|1000|800|700|2000|500|120000|2000|1000|18...,6,6,16531|700|2000|20000|50000|500
4224,20160105,92508,1,92500810,11.37,11.36,11.35,11.33,11.32,11.31,11.3,11.29,11.28,11.27,11.26,11.25,11.24,11.23,11.22,11.21,11.2,11.19,11.18,11.17,89731,157200,386700,194100,41700,86100,475921,8100,500,2000,134200,24600,800,17400,52100,55500,66000,13700,14600,400,12,12,3500|1000|800|700|2000|500|120000|2000|1000|18...,6,6,16531|700|2000|20000|50000|500
6392,20160105,92511,1,92500810,11.37,11.36,11.35,11.33,11.32,11.31,11.3,11.29,11.28,11.27,11.26,11.25,11.24,11.23,11.22,11.21,11.2,11.19,11.18,11.17,89731,157200,386700,194100,41700,86100,475921,8100,500,2000,134200,24600,800,17400,52100,55500,66000,13700,14600,400,12,12,3500|1000|800|700|2000|500|120000|2000|1000|18...,6,6,16531|700|2000|20000|50000|500
8560,20160105,92514,1,92500810,11.37,11.36,11.35,11.33,11.32,11.31,11.3,11.29,11.28,11.27,11.26,11.25,11.24,11.23,11.22,11.21,11.2,11.19,11.18,11.17,89731,157200,386700,194100,41700,86100,475921,8100,500,2000,134200,24600,800,17400,52100,55500,66000,13700,14600,400,12,12,3500|1000|800|700|2000|500|120000|2000|1000|18...,6,6,16531|700|2000|20000|50000|500
10728,20160105,92517,1,92500810,11.37,11.36,11.35,11.33,11.32,11.31,11.3,11.29,11.28,11.27,11.26,11.25,11.24,11.23,11.22,11.21,11.2,11.19,11.18,11.17,89731,157200,386700,194100,41700,86100,475921,8100,500,2000,134200,24600,800,17400,52100,55500,66000,13700,14600,400,12,12,3500|1000|800|700|2000|500|120000|2000|1000|18...,6,6,16531|700|2000|20000|50000|500
12983,20160105,92520,1,92500810,11.37,11.36,11.35,11.33,11.32,11.31,11.3,11.29,11.28,11.27,11.26,11.25,11.24,11.23,11.22,11.21,11.2,11.19,11.18,11.17,89731,157200,386700,194100,41700,86100,475921,8100,500,2000,134200,24600,800,17400,52100,55500,66000,13700,14600,400,12,12,3500|1000|800|700|2000|500|120000|2000|1000|18...,6,6,16531|700|2000|20000|50000|500
15151,20160105,92523,1,92500810,11.37,11.36,11.35,11.33,11.32,11.31,11.3,11.29,11.28,11.27,11.26,11.25,11.24,11.23,11.22,11.21,11.2,11.19,11.18,11.17,89731,157200,386700,194100,41700,86100,475921,8100,500,2000,134200,24600,800,17400,52100,55500,66000,13700,14600,400,12,12,3500|1000|800|700|2000|500|120000|2000|1000|18...,6,6,16531|700|2000|20000|50000|500
17412,20160105,92526,1,92500810,11.37,11.36,11.35,11.33,11.32,11.31,11.3,11.29,11.28,11.27,11.26,11.25,11.24,11.23,11.22,11.21,11.2,11.19,11.18,11.17,89731,157200,386700,194100,41700,86100,475921,8100,500,2000,134200,24600,800,17400,52100,55500,66000,13700,14600,400,12,12,3500|1000|800|700|2000|500|120000|2000|1000|18...,6,6,16531|700|2000|20000|50000|500
19890,20160105,92529,1,92500810,11.37,11.36,11.35,11.33,11.32,11.31,11.3,11.29,11.28,11.27,11.26,11.25,11.24,11.23,11.22,11.21,11.2,11.19,11.18,11.17,89731,157200,386700,194100,41700,86100,475921,8100,500,2000,134200,24600,800,17400,52100,55500,66000,13700,14600,400,12,12,3500|1000|800|700|2000|500|120000|2000|1000|18...,6,6,16531|700|2000|20000|50000|500


In [6]:
pd.set_option("max_columns", 200)
snapshot[(snapshot["ID"] == 2000001) & (snapshot["time"] > 93000000000)].head(50)

,date,time,clockAtArrival,ID,ordering,cum_tradesCnt,cum_volume,cum_amount,high,low,close,bid10p,bid9p,bid8p,bid7p,bid6p,bid5p,bid4p,bid3p,bid2p,bid1p,ask1p,ask2p,ask3p,ask4p,ask5p,ask6p,ask7p,ask8p,ask9p,ask10p,bid10q,bid9q,bid8q,bid7q,bid6q,bid5q,bid4q,bid3q,bid2q,bid1q,ask1q,ask2q,ask3q,ask4q,ask5q,ask6q,ask7q,ask8q,ask9q,ask10q,bid1Top1q,bid1Top2q,bid1Top3q,bid1Top4q,bid1Top5q,bid1Top6q,bid1Top7q,bid1Top8q,bid1Top9q,bid1Top10q,bid1Top11q,bid1Top12q,bid1Top13q,bid1Top14q,bid1Top15q,bid1Top16q,bid1Top17q,bid1Top18q,bid1Top19q,bid1Top20q,bid1Top21q,bid1Top22q,bid1Top23q,bid1Top24q,bid1Top25q,bid1Top26q,bid1Top27q,bid1Top28q,bid1Top29q,bid1Top30q,bid1Top31q,bid1Top32q,bid1Top33q,bid1Top34q,bid1Top35q,bid1Top36q,bid1Top37q,bid1Top38q,bid1Top39q,bid1Top40q,bid1Top41q,bid1Top42q,bid1Top43q,bid1Top44q,bid1Top45q,bid1Top46q,bid1Top47q,bid1Top48q,bid1Top49q,bid1Top50q,ask1Top1q,ask1Top2q,ask1Top3q,ask1Top4q,ask1Top5q,ask1Top6q,ask1Top7q,ask1Top8q,ask1Top9q,ask1Top10q,ask1Top11q,ask1Top12q,ask1Top13q,ask1Top14q,ask1Top15q,ask1Top16q,ask1Top17q,ask1Top18q,ask1Top19q,ask1Top20q,ask1Top21q,ask1Top22q,ask1Top23q,ask1Top24q,ask1Top25q,ask1Top26q,ask1Top27q,ask1Top28q,ask1Top29q,ask1Top30q,ask1Top31q,ask1Top32q,ask1Top33q,ask1Top34q,ask1Top35q,ask1Top36q,ask1Top37q,ask1Top38q,ask1Top39q,ask1Top40q,ask1Top41q,ask1Top42q,ask1Top43q,ask1Top44q,ask1Top45q,ask1Top46q,ask1Top47q,ask1Top48q,ask1Top49q,ask1Top50q,totalBidQ,totalAskQ,vwapBid,vwapAsk,bid1n,ask1n
157761,20160105,93000750000,1451986200750000,2000001,102,602,1840600,20726078.00,11.27,11.20,11.23,11.12,11.13,11.15,11.16,11.17,11.18,11.19,11.20,11.21,11.22,11.25,11.26,11.27,11.28,11.29,11.30,11.31,11.32,11.33,11.35,12000,25400,24700,32900,400,15600,13700,3000,6000,48600,8100,472621,76800,41700,148800,386700,158500,67931,64300,148400,46600,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,146800,500,100,1000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5156100,4466919,10.73,11.83,2,4
159323,20160105,93001790000,1451986201790000,2000001,103,645,1955500,22017084.00,11.27,11.20,11.25,11.12,11.13,11.15,11.16,11.17,11.18,11.19,11.20,11.21,11.22,11.25,11.26,11.27,11.28,11.29,11.30,11.31,11.32,11.33,11.35,12000,24900,25700,38900,400,15600,13700,12500,38500,6200,8100,409021,63700,21700,119300,386200,169100,89431,218700,112500,3300,2000,900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,101200,500,100,1000,5000,500,2100,300,1800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5209400,4507319,10.74,11.83,3,9
160885,20160105,93004660000,1451986204660000,2000001,104,825,2512100,28264899.00,11.27,11.18,11.21,11.08,11.09,11.10,11.11,11.12,11.13,11.15,11.16,11.17,11.18,11.21,11.23,11.24,11.25,11.26,11.27,11.28,11.29,11.30,11.31,51500,1600,106200,283000,21000,24900,26700,40400,900,2700,20100,115300,385300,197600,162231,537786,167600,1200,15400,13200,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5729600,5169233,10.75,11.79,1,1
162447,20160105,93012020000,1451986212020000,2000001,105,1046,3202900,36012584.00,11.27,11.15,11.22,11.12,11.13,11.14,11.15,11.16,11.17,11.18,11.19,11.20,11.22,11.23,11.24,11.25,11.26,11.27,11.28,11.29,11.30,11.31,11.32,22000,24900,3000,103800,1600,4500,7500,2000,2600,6900,51200,5400,112200,380000,198600,207531,935004,228500,42800,40000,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10000,30000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6541400,5748951,10.76,11.74,1,2
164009,20160105,93015170000,1451986215169999,2000001,106,1055,3227400,36287719.00,11.27,11.15,11.23,11.12,11.13,11.14,11.15,11.16,11.17,11.18,11.19,11.20,11.22,11.23,11.24,11.25,11.26,11.2

In [59]:
# 2017 - 2018 version
import pandas as pd
import random
import numpy as np
import glob
import os
from unrar import rarfile
import py7zr
import pickle
import datetime

columns1 = ["Date","OrigTime","SendTime","ercvtime","dbtime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID","PreClosePx",
                   "PxChnage1","PXChange2","openPrice","HighPx","LowPx","close","NumTrades","cum_volume","cum_amount","PE1","PE2","TradingPhase",
                   "totalofferqty", "vwapAsk", "totalbidqty", "vwapBid", "PreNAV", "RealTimeNAV", "WarrantPremiumRate", "UpLimitPx",
                   "DownLimitPx", "TotalLongPosition", "unknown1", "unknown2", "unknown3"]
columns2 = ['Date',"OrigTime","SendTime","ercvtime","dbtime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID",'ask1p','bid1p',
                   "ask1q","bid1q", 'ask2p','bid2p',"ask2q","bid2q",'ask3p','bid3p',"ask3q","bid3q",'ask4p','bid4p',"ask4q","bid4q",'ask5p',
                    'bid5p',"ask5q","bid5q",'ask6p','bid6p',"ask6q","bid6q",'ask7p','bid7p',"ask7q","bid7q",'ask8p','bid8p',"ask8q","bid8q",
                   'ask9p','bid9p',"ask9q","bid9q",'ask10p','bid10p',"ask10q","bid10q","bid1n","NOORDERS_B1","ORDERQTY_B1",
                    "ask1n","NOORDERS_S1","ORDERQTY_S1"]
columns3 = ["Date", "SENDTIME", "SecurityID", "DATATIMESTAMP", "PreClosePx", "openPrice", "HighPx", "LowPx", "close", "NumTrades", "cum_volume",
           "cum_amount", "TOTALLONGPOSITION", "PERATIO1", "PERATIO2", "ENDOFDAYMAKER", "totalofferqty", "vwapAsk", "totalbidqty", "vwapBid"]
columns4 = ["Date", "SENDTIME", "SecurityID", "DATATIMESTAMP", "ask10p", "ask9p", "ask8p", "ask7p", "ask6p", "ask5p", "ask4p", "ask3p", "ask2p",
           "ask1p", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid6p", "bid7p", "bid8p", "bid9p", "bid10p", "ask10q", "ask9q", "ask8q", "ask7q",
           "ask6q", "ask5q", "ask4q", "ask3q", "ask2q", "ask1q", "bid1q", "bid2q", "bid3q", "bid4q", "bid5q", "bid6q", "bid7q", "bid8q", "bid9q", 
           "bid10q", "bid1n", "NOORDERS _B1", "ORDERQTY_B1", "ask1n", "NOORDERS _S1", "ORDERQTY_S1"]
columns5 =  ["Date","OrigTime","SendTime","ercvtime","dbtime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID","PreClosePx",
                   "PxChnage1","PXChange2","openPrice","HighPx","LowPx","close","NumTrades","cum_volume","cum_amount","PE1","PE2","TradingPhase",
                   "totalofferqty", "vwapAsk", "totalbidqty", "vwapBid", "PreNAV", "RealTimeNAV", "WarrantPremiumRate", "UpLimitPx",
                   "DownLimitPx", "TotalLongPosition"]


year = "2017"
df = []
bad = []
readPath = 'J:\\LEVEL2_shenzhen\\' + year + '\\***'
dataPathLs = np.array(glob.glob(readPath))
for data in dataPathLs[:1]:
    if len(np.array(glob.glob(data +'\\***'))) == 0:
        continue
        
    print('---------------------------------------------------------------------------------------------')
    print(data)
 
    # am & pm   
    if len(np.array(glob.glob(data +'\\am_snap_level_spot.7z'))) == 1:
        date = os.path.basename(data)
        path = r'F:\SZ\2017' 
        os.chdir(data)
        try:
            a = py7zr.SevenZipFile(data + '\\am_snap_level_spot.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + '\\am_snap_level_spot.7z')
            bad.append(data + '\\am_snap_level_spot.7z')
            continue
        path1 = path + '\\' + date
        a.extractall(path = path1)
        a.close()
        try:
            a = py7zr.SevenZipFile(data + '\\pm_snap_level_spot.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + '\\pm_snap_level_spot.7z')
            bad.append(data + '\\pm_snap_level_spot.7z')
            continue
        a.extractall(path = path1)
        a.close()
        
        if len(np.array(glob.glob(path1 +'\\***_hq_snap_spot.txt'))) != 2:
            print("Less data!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            bad.append(data + '\\***_hq_snap_spot.txt')
            print(np.array(glob.glob(path1 +'\\***'))[0])
            continue
        am_snap1 = pd.read_table(path1 + "\\am_hq_snap_spot.txt", header=None)
        try:
            assert(am_snap1.shape[1] == len(columns1))
            am_snap1.columns = columns1       
        except:
            assert(am_snap1.shape[1] == len(columns5))
            am_snap1.columns = columns5     
        pm_snap1 = pd.read_table(path1 + "\\pm_hq_snap_spot.txt", header=None)
        try:
            assert(pm_snap1.shape[1] == len(columns1))
            pm_snap1.columns = columns1       
        except:
            assert(pm_snap1.shape[1] == len(columns5))
            pm_snap1.columns = columns5     
        snapshot1 = pd.concat([am_snap1, pm_snap1]).sort_values(by=["SecurityID", "OrigTime"])
        del am_snap1
        del pm_snap1
        
        am_snap1 = pd.read_table(path1 + "\\am_snap_level_spot.txt", header=None)
        assert(am_snap1.shape[1] == len(columns2))
        am_snap1.columns = columns2       
        pm_snap1 = pd.read_table(path1 + "\\pm_snap_level_spot.txt", header=None)
        assert(pm_snap1.shape[1] == len(columns2))
        pm_snap1.columns = columns2       
        snapshot2 = pd.concat([am_snap1, pm_snap1]).sort_values(by=["SecurityID", "OrigTime"])
        del am_snap1
        del pm_snap1
        
        snapshot1 = snapshot1[(snapshot1["SecurityID"] < 4000) | (snapshot1["SecurityID"] > 300000)]
        snapshot2 = snapshot2[(snapshot2["SecurityID"] < 4000) | (snapshot2["SecurityID"] > 300000)]
        snapshot1['time'] = (snapshot1['OrigTime'] - int(snapshot1['OrigTime'].iloc[0]//1000000000*1000000000)).astype(int)
        snapshot2['time'] = (snapshot2['OrigTime'] - int(snapshot2['OrigTime'].iloc[0]//1000000000*1000000000)).astype(int)
        snapshot = pd.merge(snapshot1, snapshot2, on=['Date',"OrigTime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID", "time"], how="outer")
        try:
            assert((snapshot.shape[0] == snapshot1.shape[0]) & (snapshot.shape[0] == snapshot2.shape[0]))
        except:
            if snapshot.shape[0] == snapshot1.shape[0]:
                print("snapshot1 have more ticks than snapshot2")
                if all(snapshot[(snapshot["dbtime_y"].isnull()) & ((snapshot["SecurityID"] < 4000) | (snapshot["SecurityID"] > 200000))]["time"].unique() > 150000000):
                    print("More ticks happens after 15:00")
                elif all(snapshot[(snapshot["dbtime_y"].isnull()) & ((snapshot["SecurityID"] < 4000) | (snapshot["SecurityID"] > 200000))]["time"].unique() < 93000000):
                    print("More ticks happens before 9:30")
                else:
                    print("There are ticks happens before 15:00, after 9:30")
            if snapshot.shape[0] == snapshot2.shape[0]:
                print("snapshot2 have more ticks than snapshot1")
                if all(snapshot[(snapshot["dbtime_x"].isnull()) & ((snapshot["SecurityID"] < 4000) | (snapshot["SecurityID"] > 200000))]["time"].unique() > 150000000):
                    print("More ticks happens after 15:00")
                elif all(snapshot[(snapshot["dbtime_x"].isnull()) & ((snapshot["SecurityID"] < 4000) | (snapshot["SecurityID"] > 200000))]["time"].unique() < 93000000):
                    print("More ticks happens before 9:30")
                else:
                    print("There are ticks happens before 15:00, after 9:30")
            else:
                print("snapshot2 don't join with snapshot1")
            snapshot = pd.merge(snapshot1, snapshot2, on=['Date',"OrigTime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID", "time"])
        del snapshot1
        del snapshot2
           
        snapshot = snapshot.rename(columns={"SecurityID":"StockID"})
        print(int(snapshot['OrigTime'].iloc[0]//1000000000*1000000000))
        
        print("am & pm data 未分卷")
        
    elif len(np.array(glob.glob(data +'\\am_snap_level_spot.7z.001'))) == 1:    
        date = os.path.basename(data)
        path = r'F:\SZ\2017' 
        os.chdir(data)
        os.system("copy /b am_snap_level_spot.7z.* am_snap_level_spot.7z")
        try:
            a = py7zr.SevenZipFile(data + '\\am_snap_level_spot.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + '\\am_snap_level_spot.7z')
            bad.append(data + '\\am_snap_level_spot.7z')
            continue
        path1 = path + '\\' + date
        a.extractall(path = path1)
        a.close()
        
        os.system("copy /b pm_snap_level_spot.7z.* pm_snap_level_spot.7z")
        try:
            a = py7zr.SevenZipFile(data + '\\pm_snap_level_spot.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + '\\pm_snap_level_spot.7z')
            bad.append(data + '\\pm_snap_level_spot.7z')
            continue
        a.extractall(path = path1)
        a.close()
        
        if len(np.array(glob.glob(path1 +'\\***_hq_snap_spot.txt'))) != 2:
            print("Less data!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            bad.append(data + '\\***_hq_snap_spot.txt')
            print(np.array(glob.glob(path1 +'\\***'))[0])
            continue
        am_snap1 = pd.read_table(path1 + "\\am_hq_snap_spot.txt", header=None)
        try:
            assert(am_snap1.shape[1] == len(columns1))
            am_snap1.columns = columns1       
        except:
            assert(am_snap1.shape[1] == len(columns5))
            am_snap1.columns = columns5       
        pm_snap1 = pd.read_table(path1 + "\\pm_hq_snap_spot.txt", header=None)
        try:
            assert(pm_snap1.shape[1] == len(columns1))
            pm_snap1.columns = columns1       
        except:
            assert(pm_snap1.shape[1] == len(columns5))
            pm_snap1.columns = columns5     
        snapshot1 = pd.concat([am_snap1, pm_snap1]).sort_values(by=["SecurityID", "OrigTime"])
        del am_snap1
        del pm_snap1
        
        am_snap1 = pd.read_table(path1 + "\\am_snap_level_spot.txt", header=None)
        assert(am_snap1.shape[1] == len(columns2))
        am_snap1.columns = columns2        
        pm_snap1 = pd.read_table(path1 + "\\pm_snap_level_spot.txt", header=None)
        assert(pm_snap1.shape[1] == len(columns2))
        pm_snap1.columns = columns2       
        snapshot2 = pd.concat([am_snap1, pm_snap1]).sort_values(by=["SecurityID", "OrigTime"])
        del am_snap1
        del pm_snap1
        
        snapshot1 = snapshot1[(snapshot1["SecurityID"] < 4000) | (snapshot1["SecurityID"] > 300000)]
        snapshot2 = snapshot2[(snapshot2["SecurityID"] < 4000) | (snapshot2["SecurityID"] > 300000)]
        snapshot1['time'] = (snapshot1['OrigTime'] - int(snapshot1['OrigTime'].iloc[0]//1000000000*1000000000)).astype(int)
        snapshot2['time'] = (snapshot2['OrigTime'] - int(snapshot2['OrigTime'].iloc[0]//1000000000*1000000000)).astype(int)
        snapshot = pd.merge(snapshot1, snapshot2, on=['Date',"OrigTime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID", "time"], how="outer")
        try:
            assert((snapshot.shape[0] == snapshot1.shape[0]) & (snapshot.shape[0] == snapshot2.shape[0]))
        except:
            if snapshot.shape[0] == snapshot1.shape[0]:
                print("snapshot1 have more ticks than snapshot2")
                if all(snapshot[(snapshot["dbtime_y"].isnull()) & ((snapshot["SecurityID"] < 4000) | (snapshot["SecurityID"] > 200000))]["time"].unique() > 150000000):
                    print("More ticks happens after 15:00")
                elif all(snapshot[(snapshot["dbtime_y"].isnull()) & ((snapshot["SecurityID"] < 4000) | (snapshot["SecurityID"] > 200000))]["time"].unique() < 93000000):
                    print("More ticks happens before 9:30")
                else:
                    print("There are ticks happens before 15:00, after 9:30")
            if snapshot.shape[0] == snapshot2.shape[0]:
                print("snapshot2 have more ticks than snapshot1")
                if all(snapshot[(snapshot["dbtime_x"].isnull()) & ((snapshot["SecurityID"] < 4000) | (snapshot["SecurityID"] > 200000))]["time"].unique() > 150000000):
                    print("More ticks happens after 15:00")
                elif all(snapshot[(snapshot["dbtime_x"].isnull()) & ((snapshot["SecurityID"] < 4000) | (snapshot["SecurityID"] > 200000))]["time"].unique() < 93000000):
                    print("More ticks happens before 9:30")
                else:
                    print("There are ticks happens before 15:00, after 9:30")
            else:
                print("snapshot2 don't join with snapshot1")
            snapshot = pd.merge(snapshot1, snapshot2, on=['Date',"OrigTime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID", "time"])
        del snapshot1
        del snapshot2
        
        snapshot = snapshot.rename(columns={"SecurityID":"StockID"})
        print(int(snapshot['OrigTime'].iloc[0]//1000000000*1000000000))
        
        print("am & pm data 分卷") 
   

    elif len(np.array(glob.glob(data +'\\snap_level.7z'))) == 1: 
        date = os.path.basename(data)
        path = r'F:\SZ\2017' 
        os.chdir(data)
        try:
            a = py7zr.SevenZipFile(data + '\\snap_level.7z','r',filters=None)
        except:
            print("Bad unzip here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(data + '\\snap_level.7z')
            bad.append(data + '\\snap_level.7z')
            continue
        path1 = path + '\\' + date
        a.extractall(path = path1)
        a.close()
        
        if len(np.array(glob.glob(path1 +'\\hq_snap.txt'))) != 1:
            print("Less data!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            bad.append(data + '\\hq_snap.txt')
            print(np.array(glob.glob(path1 +'\\***'))[0])
            continue
        snapshot1 = pd.read_table(path1 + "\\hq_snap.txt", header=None, encoding="UTF-8-sig")
        try:
            assert(snapshot1.shape[1] == len(columns1))
            snapshot1.columns = columns1
        except:
            assert(snapshot1.shape[1] == len(columns5))
            snapshot1.columns = columns5
        
        snapshot1["SecurityID"] = snapshot1["SecurityID"].astype(int)
        snapshot2["SecurityID"] = snapshot2["SecurityID"].astype(int)
        snapshot1 = snapshot1[(snapshot1["SecurityID"] < 4000) | (snapshot1["SecurityID"] > 300000)]
        snapshot2 = snapshot2[(snapshot2["SecurityID"] < 4000) | (snapshot2["SecurityID"] > 300000)]
        snapshot1['time'] = (snapshot1['OrigTime'] - int(snapshot1['OrigTime'].iloc[0]//1000000000*1000000000)).astype(np.int64)
        snapshot2['time'] = (snapshot2['OrigTime'] - int(snapshot2['OrigTime'].iloc[0]//1000000000*1000000000)).astype(np.int64)
        
        snapshot = pd.merge(snapshot1, snapshot2, on=['Date',"OrigTime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID", "time"], how="outer")
        try:
            assert((snapshot.shape[0] == snapshot1.shape[0]) & (snapshot.shape[0] == snapshot2.shape[0]))
        except:
            if snapshot.shape[0] == snapshot1.shape[0]:
                print("snapshot1 have more ticks than snapshot2")
                if all(snapshot[(snapshot["dbtime_y"].isnull()) & ((snapshot["SecurityID"] < 4000) | (snapshot["SecurityID"] > 200000))]["time"].unique() > 150000000):
                    print("More ticks happens after 15:00")
                elif all(snapshot[(snapshot["dbtime_y"].isnull()) & ((snapshot["SecurityID"] < 4000) | (snapshot["SecurityID"] > 200000))]["time"].unique() < 93000000):
                    print("More ticks happens before 9:30")
                else:
                    print("There are ticks happens before 15:00, after 9:30")
            if snapshot.shape[0] == snapshot2.shape[0]:
                print("snapshot2 have more ticks than snapshot1")
                if all(snapshot[(snapshot["dbtime_x"].isnull()) & ((snapshot["SecurityID"] < 4000) | (snapshot["SecurityID"] > 200000))]["time"].unique() > 150000000):
                    print("More ticks happens after 15:00")
                elif all(snapshot[(snapshot["dbtime_x"].isnull()) & ((snapshot["SecurityID"] < 4000) | (snapshot["SecurityID"] > 200000))]["time"].unique() < 93000000):
                    print("More ticks happens before 9:30")
                else:
                    print("There are ticks happens before 15:00, after 9:30")
            else:
                print("snapshot2 don't join with snapshot1")
            snapshot = pd.merge(snapshot1, snapshot2, on=['Date',"OrigTime","ChannelNo","SecurityID","SecurityIDsource", "MDStreamID", "time"])
        del snapshot1
        del snapshot2
        
        snapshot = snapshot.rename(columns={"SecurityID":"StockID"})
        print(int(snapshot['OrigTime'].iloc[0]//1000000000*1000000000))
        
        print("深交所数据")
        
        
    snapshot["clockAtArrival"] = snapshot["OrigTime"].astype(str).apply(lambda x: np.int64(datetime.datetime.strptime(x, '%Y%m%d%H%M%S%f').timestamp()*1e6))
    snapshot = snapshot.rename(columns={"Date":"date", "NumTrades":"cum_tradesCnt", "HighPx":"high", "LowPx":"low", "totalofferqty":
                                   "totalAskQ", "totalbidqty":"totalBidQ"})
    snapshot["ID"] = snapshot["StockID"] + 2000000
    snapshot["ordering"] = snapshot.groupby("ID").cumcount()
    snapshot["ordering"] = snapshot["ordering"] + 1
    snapshot["time"] = snapshot["time"].astype(np.int64) * 1000
    
    snapshot.loc[(~snapshot["ORDERQTY_B1"].isnull()) & (snapshot["ORDERQTY_B1"].str.contains('\|')), "ORDERQTY_B1"]=snapshot[(~snapshot["ORDERQTY_B1"].isnull()) & (snapshot["ORDERQTY_B1"].str.contains('\|'))]["ORDERQTY_B1"].apply(lambda x: x.split('|'))
    snapshot.loc[(~snapshot["ORDERQTY_S1"].isnull()) & (snapshot["ORDERQTY_S1"].str.contains('\|')), "ORDERQTY_S1"]=snapshot[(~snapshot["ORDERQTY_S1"].isnull()) & (snapshot["ORDERQTY_S1"].str.contains('\|'))]["ORDERQTY_S1"].apply(lambda x: x.split('|'))
    for i in range(1, 51):
        snapshot["bid1Top" + str(i) + 'q'] = '0'
        snapshot["ask1Top" + str(i) + 'q'] = '0'
    for i in range(1, 51):
        snapshot.loc[i <= snapshot["bid1n"], "bid1Top" + str(i) + 'q'] = snapshot.loc[i <= snapshot["bid1n"], "ORDERQTY_B1"].apply(lambda x: x[i-1])
        snapshot.loc[i <= snapshot["ask1n"], "ask1Top" + str(i) + 'q'] = snapshot.loc[i <= snapshot["ask1n"], "ORDERQTY_S1"].apply(lambda x: x[i-1])    
    
    for columns in ["cum_tradesCnt", "cum_volume", 'ask1q','ask2q','ask3q','ask4q','ask5q','ask6q','ask7q','ask8q','ask9q',
                        'ask10q', 'bid1q','bid2q','bid3q','bid4q','bid5q','bid6q','bid7q','bid8q','bid9q','bid10q', 'bid1Top1q','bid1Top2q','bid1Top3q','bid1Top4q','bid1Top5q','bid1Top6q',
        'bid1Top7q','bid1Top8q','bid1Top9q','bid1Top10q','bid1Top11q','bid1Top12q','bid1Top13q','bid1Top14q','bid1Top15q','bid1Top16q','bid1Top17q','bid1Top18q',
        'bid1Top19q','bid1Top20q','bid1Top21q','bid1Top22q','bid1Top23q','bid1Top24q','bid1Top25q','bid1Top26q','bid1Top27q','bid1Top28q','bid1Top29q',
        'bid1Top30q','bid1Top31q','bid1Top32q','bid1Top33q','bid1Top34q','bid1Top35q','bid1Top36q','bid1Top37q','bid1Top38q','bid1Top39q','bid1Top40q',
        'bid1Top41q','bid1Top42q','bid1Top43q','bid1Top44q','bid1Top45q','bid1Top46q','bid1Top47q','bid1Top48q','bid1Top49q','bid1Top50q', 'ask1Top1q',
        'ask1Top2q','ask1Top3q','ask1Top4q','ask1Top5q','ask1Top6q','ask1Top7q','ask1Top8q','ask1Top9q','ask1Top10q','ask1Top11q','ask1Top12q','ask1Top13q',
        'ask1Top14q','ask1Top15q','ask1Top16q','ask1Top17q','ask1Top18q','ask1Top19q','ask1Top20q','ask1Top21q','ask1Top22q','ask1Top23q',
        'ask1Top24q','ask1Top25q','ask1Top26q','ask1Top27q','ask1Top28q','ask1Top29q','ask1Top30q','ask1Top31q','ask1Top32q','ask1Top33q',
        'ask1Top34q','ask1Top35q','ask1Top36q','ask1Top37q','ask1Top38q','ask1Top39q','ask1Top40q','ask1Top41q','ask1Top42q','ask1Top43q',
        'ask1Top44q','ask1Top45q','ask1Top46q','ask1Top47q','ask1Top48q','ask1Top49q','ask1Top50q', "totalBidQ", "totalAskQ", "bid1n", "ask1n"]:
            snapshot[columns] = snapshot[columns].astype(int)
    snapshot = snapshot[["date", "time", "clockAtArrival", "ID", "ordering", "cum_tradesCnt", "cum_volume", "cum_amount",
                            "high", "low", "close", 'bid10p','bid9p','bid8p','bid7p','bid6p','bid5p','bid4p','bid3p','bid2p','bid1p',
                            'ask1p','ask2p','ask3p','ask4p','ask5p','ask6p','ask7p','ask8p','ask9p','ask10p', 'bid10q','bid9q','bid8q',
                             'bid7q','bid6q','bid5q','bid4q','bid3q','bid2q','bid1q', 'ask1q','ask2q','ask3q','ask4q','ask5q','ask6q',
                             'ask7q','ask8q','ask9q','ask10q', 'bid1Top1q','bid1Top2q','bid1Top3q','bid1Top4q','bid1Top5q','bid1Top6q',
        'bid1Top7q','bid1Top8q','bid1Top9q','bid1Top10q','bid1Top11q','bid1Top12q','bid1Top13q','bid1Top14q','bid1Top15q','bid1Top16q','bid1Top17q','bid1Top18q',
        'bid1Top19q','bid1Top20q','bid1Top21q','bid1Top22q','bid1Top23q','bid1Top24q','bid1Top25q','bid1Top26q','bid1Top27q','bid1Top28q','bid1Top29q',
        'bid1Top30q','bid1Top31q','bid1Top32q','bid1Top33q','bid1Top34q','bid1Top35q','bid1Top36q','bid1Top37q','bid1Top38q','bid1Top39q','bid1Top40q',
        'bid1Top41q','bid1Top42q','bid1Top43q','bid1Top44q','bid1Top45q','bid1Top46q','bid1Top47q','bid1Top48q','bid1Top49q','bid1Top50q', 'ask1Top1q',
        'ask1Top2q','ask1Top3q','ask1Top4q','ask1Top5q','ask1Top6q','ask1Top7q','ask1Top8q','ask1Top9q','ask1Top10q','ask1Top11q','ask1Top12q','ask1Top13q',
        'ask1Top14q','ask1Top15q','ask1Top16q','ask1Top17q','ask1Top18q','ask1Top19q','ask1Top20q','ask1Top21q','ask1Top22q','ask1Top23q',
        'ask1Top24q','ask1Top25q','ask1Top26q','ask1Top27q','ask1Top28q','ask1Top29q','ask1Top30q','ask1Top31q','ask1Top32q','ask1Top33q',
        'ask1Top34q','ask1Top35q','ask1Top36q','ask1Top37q','ask1Top38q','ask1Top39q','ask1Top40q','ask1Top41q','ask1Top42q','ask1Top43q',
        'ask1Top44q','ask1Top45q','ask1Top46q','ask1Top47q','ask1Top48q','ask1Top49q','ask1Top50q',"totalBidQ", "totalAskQ","vwapBid", "vwapAsk",
                        "bid1n", "ask1n"]]

print(bad)    

    

---------------------------------------------------------------------------------------------
J:\LEVEL2_shenzhen\2017\0103
20170103000000000
am & pm data 未分卷
[]


In [2]:
pd.set_option("max_columns", 200)
snapshot[(snapshot["ID"] == 2000001) & (snapshot["time"] > 92500000000)].head(50)

,date,time,clockAtArrival,ID,ordering,cum_tradesCnt,cum_volume,cum_amount,high,low,close,bid10p,bid9p,bid8p,bid7p,bid6p,bid5p,bid4p,bid3p,bid2p,bid1p,ask1p,ask2p,ask3p,ask4p,ask5p,ask6p,ask7p,ask8p,ask9p,ask10p,bid10q,bid9q,bid8q,bid7q,bid6q,bid5q,bid4q,bid3q,bid2q,bid1q,ask1q,ask2q,ask3q,ask4q,ask5q,ask6q,ask7q,ask8q,ask9q,ask10q,bid1Top1q,bid1Top2q,bid1Top3q,bid1Top4q,bid1Top5q,bid1Top6q,bid1Top7q,bid1Top8q,bid1Top9q,bid1Top10q,bid1Top11q,bid1Top12q,bid1Top13q,bid1Top14q,bid1Top15q,bid1Top16q,bid1Top17q,bid1Top18q,bid1Top19q,bid1Top20q,bid1Top21q,bid1Top22q,bid1Top23q,bid1Top24q,bid1Top25q,bid1Top26q,bid1Top27q,bid1Top28q,bid1Top29q,bid1Top30q,bid1Top31q,bid1Top32q,bid1Top33q,bid1Top34q,bid1Top35q,bid1Top36q,bid1Top37q,bid1Top38q,bid1Top39q,bid1Top40q,bid1Top41q,bid1Top42q,bid1Top43q,bid1Top44q,bid1Top45q,bid1Top46q,bid1Top47q,bid1Top48q,bid1Top49q,bid1Top50q,ask1Top1q,ask1Top2q,ask1Top3q,ask1Top4q,ask1Top5q,ask1Top6q,ask1Top7q,ask1Top8q,ask1Top9q,ask1Top10q,ask1Top11q,ask1Top12q,ask1Top13q,ask1Top14q,ask1Top15q,ask1Top16q,ask1Top17q,ask1Top18q,ask1Top19q,ask1Top20q,ask1Top21q,ask1Top22q,ask1Top23q,ask1Top24q,ask1Top25q,ask1Top26q,ask1Top27q,ask1Top28q,ask1Top29q,ask1Top30q,ask1Top31q,ask1Top32q,ask1Top33q,ask1Top34q,ask1Top35q,ask1Top36q,ask1Top37q,ask1Top38q,ask1Top39q,ask1Top40q,ask1Top41q,ask1Top42q,ask1Top43q,ask1Top44q,ask1Top45q,ask1Top46q,ask1Top47q,ask1Top48q,ask1Top49q,ask1Top50q,totalBidQ,totalAskQ,vwapBid,vwapAsk,bid1n,ask1n
66,20190102,92503000000,1546392303000000,2000001,67,157,389500,3657405.00,9.39,9.39,9.39,9.30,9.31,9.32,9.33,9.34,9.35,9.36,9.37,9.38,9.39,9.40,9.41,9.42,9.43,9.44,9.45,9.46,9.47,9.48,9.49,231400,169300,25800,13700,11000,43700,67700,65100,79600,183300,13400,58900,19100,1300,17800,55100,19900,22400,104300,50700,64400,200,100,1000,5000,1000,100,900,1400,200,100000,2000,2000,2000,2500,500,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,400,10000,3000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1793100,3064057,9.10,9.84,16,3
67,20190102,92603000000,1546392363000000,2000001,68,157,389500,3657405.00,9.39,9.39,9.39,9.30,9.31,9.32,9.33,9.34,9.35,9.36,9.37,9.38,9.39,9.40,9.41,9.42,9.43,9.44,9.45,9.46,9.47,9.48,9.49,231400,169300,25800,13700,11000,43700,67700,65100,79600,183300,13400,58900,19100,1300,17800,55100,19900,22400,104300,50700,64400,200,100,1000,5000,1000,100,900,1400,200,100000,2000,2000,2000,2500,500,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,400,10000,3000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1793100,3064057,9.10,9.84,16,3
68,20190102,92703000000,1546392423000000,2000001,69,157,389500,3657405.00,9.39,9.39,9.39,9.30,9.31,9.32,9.33,9.34,9.35,9.36,9.37,9.38,9.39,9.40,9.41,9.42,9.43,9.44,9.45,9.46,9.47,9.48,9.49,231400,169300,25800,13700,11000,43700,67700,65100,79600,183300,13400,58900,19100,1300,17800,55100,19900,22400,104300,50700,64400,200,100,1000,5000,1000,100,900,1400,200,100000,2000,2000,2000,2500,500,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,400,10000,3000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1793100,3064057,9.10,9.84,16,3
69,20190102,92803000000,1546392483000000,2000001,70,157,389500,3657405.00,9.39,9.39,9.39,9.30,9.31,9.32,9.33,9.34,9.35,9.36,9.37,9.38,9.39,9.40,9.41,9.42,9.43,9.44,9.45,9.46,9.47,9.48,9.49,231400,169300,25800,13700,11000,43700,67700,65100,79600,183300,13400,58900,19100,1300,17800,55100,19900,22400,104300,50700,64400,200,100,1000,5000,1000,100,900,1400,200,100000,2000,2000,2000,2500,500,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,400,10000,3000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1793100,3064057,9.10,9.84,16,3
70,20190102,92903000000,1546392543000000,2000001,71,157,389500,3657405.00,9.39,9.39,9.39,9.30,9.31,9.32,9.33,9.34,9.35,9.36,9.37,9.38,9.39,9.40,9.41,

In [66]:
datetime.datetime.fromtimestamp(1483407114000000/1e6).strftime("%Y-%m-%d %H:%M:%S %f")

'2017-01-03 09:31:54'

In [1]:
# 2019 version
import pandas as pd
import random
import numpy as np
import glob
import os
from unrar import rarfile
import py7zr
import pickle
import datetime

year = "2019"
df = []
bad = []
readPath = 'E:\\SZ\\' + year + '\\***'
dataPathLs = np.array(glob.glob(readPath))
for data in dataPathLs[:1]:
    if len(np.array(glob.glob(data +'\\***.txt'))) != 4:
        print("Less data!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        print(data + '\\Snapshot.pkl')
        bad.append(data + '\\Snapshot.pkl')
        continue
    
    print('---------------------------------------------------------------------------------------------')
    print(data)
    
    F1 = open(data+"\\Snapshot.pkl", 'rb')
    snapshot = pickle.load(F1)
    snapshot['time'] = (snapshot['OrigTime'] - int(snapshot["OrigTime"].iloc[0]//1000000000 * 1000000000)).astype(int)
 
    
    snapshot["clockAtArrival"] = snapshot["OrigTime"].astype(str).apply(lambda x: np.int64(datetime.datetime.strptime(x, '%Y%m%d%H%M%S%f').timestamp()*1e6))
    snapshot = snapshot.rename(columns={"Date":"date", "NumTrades":"cum_tradesCnt", "HighPx":"high", "LowPx":"low", "totalofferqty":
                                   "totalAskQuantity", "totalbidqty":"totalBidQuantity", "NUMORDERS_B1":"bid1n", "NUMORDERS_S1":"ask1n",
                                    "wa_offerPrice": "vwapAsk", "wa_bidPrice":"vwapBid", "PreClosePx":"prevClose", "openPrice":"open"})
    snapshot["ID"] = snapshot["StockID"] + 2000000
    snapshot["ordering"] = snapshot.groupby("ID").cumcount()
    snapshot["ordering"] = snapshot["ordering"] + 1
    snapshot["time"] = snapshot["time"].astype(np.int64) * 1000
    
    snapshot.loc[(~snapshot["ORDERQTY_B1"].isnull()), "ORDERQTY_B1"]=snapshot[(~snapshot["ORDERQTY_B1"].isnull())]["ORDERQTY_B1"].apply(lambda x: [int(i) for i in x.split('|')])
    snapshot.loc[(~snapshot["ORDERQTY_S1"].isnull()), "ORDERQTY_S1"]=snapshot[(~snapshot["ORDERQTY_S1"].isnull())]["ORDERQTY_S1"].apply(lambda x: [int(i) for i in x.split('|')])
    ## lambda x: [int(i) for i in x.split('|')]
    for i in range(1, 51):
        snapshot["bid1Top" + str(i) + 'q'] = 0
        snapshot["ask1Top" + str(i) + 'q'] = 0
    for i in range(1, 51):
        snapshot.loc[i <= snapshot["bid1n"], "bid1Top" + str(i) + 'q'] = snapshot.loc[i <= snapshot["bid1n"], "ORDERQTY_B1"].apply(lambda x: x[i-1])
        snapshot.loc[i <= snapshot["ask1n"], "ask1Top" + str(i) + 'q'] = snapshot.loc[i <= snapshot["ask1n"], "ORDERQTY_S1"].apply(lambda x: x[i-1])    


    for columns in ["cum_tradesCnt", "cum_volume", 'ask1q','ask2q','ask3q','ask4q','ask5q','ask6q','ask7q','ask8q','ask9q',
                        'ask10q', 'bid1q','bid2q','bid3q','bid4q','bid5q','bid6q','bid7q','bid8q','bid9q','bid10q',
                    "totalBidQuantity", "totalAskQuantity", "bid1n", "ask1n"]:
        snapshot[columns] = snapshot[columns].astype('int64')

        
    
    for cols in ["prevClose", 'open', "high", "low", "close", 'bid10p','bid9p','bid8p','bid7p','bid6p','bid5p','bid4p','bid3p',
             'bid2p','bid1p','ask1p','ask2p','ask3p','ask4p','ask5p','ask6p','ask7p','ask8p','ask9p','ask10p', 'vwapBid', "vwapAsk"]:
        snapshot[cols] = (snapshot[cols] * 10000).round(0).astype('int64') 
    for cols in ["cum_amount"]:
        snapshot[cols] = (snapshot[cols] * 10000).round(0).astype('int64')
    snapshot['datetime'] = snapshot["clockAtArrival"].apply(lambda x: datetime.datetime.fromtimestamp(x/1e6))
    for cols in ['bid10n', 'bid9n', 'bid8n', 'bid7n', 'bid6n', 'bid5n', 'bid4n', 'bid3n', 'bid2n', 'ask2n', 'ask3n', 'ask4n', 'ask5n', 'ask6n','ask7n', 
                 'ask8n', 'ask9n', 'ask10n', "totalBidOrders",'totalAskOrders','totalBidLevels', 'totalAskLevels', 'bidTradeMaxDuration', 'askTradeMaxDuration', 'cum_canceledBuyOrders', 'cum_canceledBuyVolume',
        "cum_canceledBuyAmount", "cum_canceledSellOrders", 'cum_canceledSellVolume',"cum_canceledSellAmount"]:
        snapshot[cols] = 0
    
    snapshot = snapshot[["date", "time", "clockAtArrival", "datetime", "ID", "ordering", "cum_tradesCnt", "cum_volume", "cum_amount", "prevClose",
                            "open", "high", "low", "close", 'bid10p','bid9p','bid8p','bid7p','bid6p','bid5p','bid4p','bid3p','bid2p','bid1p',
                            'ask1p','ask2p','ask3p','ask4p','ask5p','ask6p','ask7p','ask8p','ask9p','ask10p', 'bid10q','bid9q','bid8q',
                             'bid7q','bid6q','bid5q','bid4q','bid3q','bid2q','bid1q', 'ask1q','ask2q','ask3q','ask4q','ask5q','ask6q',
                             'ask7q','ask8q','ask9q','ask10q', 'bid10n', 'bid9n', 'bid8n', 'bid7n', 'bid6n', 'bid5n', 'bid4n', 'bid3n', 'bid2n', 'bid1n', 
                             'ask1n', 'ask2n', 'ask3n', 'ask4n', 'ask5n', 'ask6n','ask7n', 'ask8n', 'ask9n', 'ask10n','bid1Top1q','bid1Top2q','bid1Top3q','bid1Top4q','bid1Top5q','bid1Top6q',
        'bid1Top7q','bid1Top8q','bid1Top9q','bid1Top10q','bid1Top11q','bid1Top12q','bid1Top13q','bid1Top14q','bid1Top15q','bid1Top16q','bid1Top17q','bid1Top18q',
        'bid1Top19q','bid1Top20q','bid1Top21q','bid1Top22q','bid1Top23q','bid1Top24q','bid1Top25q','bid1Top26q','bid1Top27q','bid1Top28q','bid1Top29q',
        'bid1Top30q','bid1Top31q','bid1Top32q','bid1Top33q','bid1Top34q','bid1Top35q','bid1Top36q','bid1Top37q','bid1Top38q','bid1Top39q','bid1Top40q',
        'bid1Top41q','bid1Top42q','bid1Top43q','bid1Top44q','bid1Top45q','bid1Top46q','bid1Top47q','bid1Top48q','bid1Top49q','bid1Top50q', 'ask1Top1q',
        'ask1Top2q','ask1Top3q','ask1Top4q','ask1Top5q','ask1Top6q','ask1Top7q','ask1Top8q','ask1Top9q','ask1Top10q','ask1Top11q','ask1Top12q','ask1Top13q',
        'ask1Top14q','ask1Top15q','ask1Top16q','ask1Top17q','ask1Top18q','ask1Top19q','ask1Top20q','ask1Top21q','ask1Top22q','ask1Top23q',
        'ask1Top24q','ask1Top25q','ask1Top26q','ask1Top27q','ask1Top28q','ask1Top29q','ask1Top30q','ask1Top31q','ask1Top32q','ask1Top33q',
        'ask1Top34q','ask1Top35q','ask1Top36q','ask1Top37q','ask1Top38q','ask1Top39q','ask1Top40q','ask1Top41q','ask1Top42q','ask1Top43q',
        'ask1Top44q','ask1Top45q','ask1Top46q','ask1Top47q','ask1Top48q','ask1Top49q','ask1Top50q',"totalBidQuantity", "totalAskQuantity","vwapBid", "vwapAsk",
        "totalBidOrders",'totalAskOrders','totalBidLevels', 'totalAskLevels', 'bidTradeMaxDuration', 'askTradeMaxDuration', 'cum_canceledBuyOrders', 'cum_canceledBuyVolume',
        "cum_canceledBuyAmount", "cum_canceledSellOrders", 'cum_canceledSellVolume',"cum_canceledSellAmount"]]

print(bad)    

    

---------------------------------------------------------------------------------------------
E:\SZ\2019\0102
[]


In [3]:
pd.set_option("max_columns", 200)
snapshot[(snapshot["ID"] == 2000001)].head(20)

,date,time,clockAtArrival,ID,ordering,cum_tradesCnt,cum_volume,cum_amount,high,low,close,bid10p,bid9p,bid8p,bid7p,bid6p,bid5p,bid4p,bid3p,bid2p,bid1p,ask1p,ask2p,ask3p,ask4p,ask5p,ask6p,ask7p,ask8p,ask9p,ask10p,bid10q,bid9q,bid8q,bid7q,bid6q,bid5q,bid4q,bid3q,bid2q,bid1q,ask1q,ask2q,ask3q,ask4q,ask5q,ask6q,ask7q,ask8q,ask9q,ask10q,bid1Top1q,bid1Top2q,bid1Top3q,bid1Top4q,bid1Top5q,bid1Top6q,bid1Top7q,bid1Top8q,bid1Top9q,bid1Top10q,bid1Top11q,bid1Top12q,bid1Top13q,bid1Top14q,bid1Top15q,bid1Top16q,bid1Top17q,bid1Top18q,bid1Top19q,bid1Top20q,bid1Top21q,bid1Top22q,bid1Top23q,bid1Top24q,bid1Top25q,bid1Top26q,bid1Top27q,bid1Top28q,bid1Top29q,bid1Top30q,bid1Top31q,bid1Top32q,bid1Top33q,bid1Top34q,bid1Top35q,bid1Top36q,bid1Top37q,bid1Top38q,bid1Top39q,bid1Top40q,bid1Top41q,bid1Top42q,bid1Top43q,bid1Top44q,bid1Top45q,bid1Top46q,bid1Top47q,bid1Top48q,bid1Top49q,bid1Top50q,ask1Top1q,ask1Top2q,ask1Top3q,ask1Top4q,ask1Top5q,ask1Top6q,ask1Top7q,ask1Top8q,ask1Top9q,ask1Top10q,ask1Top11q,ask1Top12q,ask1Top13q,ask1Top14q,ask1Top15q,ask1Top16q,ask1Top17q,ask1Top18q,ask1Top19q,ask1Top20q,ask1Top21q,ask1Top22q,ask1Top23q,ask1Top24q,ask1Top25q,ask1Top26q,ask1Top27q,ask1Top28q,ask1Top29q,ask1Top30q,ask1Top31q,ask1Top32q,ask1Top33q,ask1Top34q,ask1Top35q,ask1Top36q,ask1Top37q,ask1Top38q,ask1Top39q,ask1Top40q,ask1Top41q,ask1Top42q,ask1Top43q,ask1Top44q,ask1Top45q,ask1Top46q,ask1Top47q,ask1Top48q,ask1Top49q,ask1Top50q,totalBidQ,totalAskQ,vwapBid,vwapAsk,bid1n,ask1n
0,20190102,90000000000,1546390800000000,2000001,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0
1,20190102,90100000000,1546390860000000,2000001,2,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0
2,20190102,90200000000,1546390920000000,2000001,3,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0
3,20190102,90300000000,1546390980000000,2000001,4,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0
4,20190102,90400000000,1546391040000000,2000001,5,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0
5,20190102,90500000000,1546391100000000,2000001,6,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0
6,20190102,90600000000,1546391160000000,2000001,7,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0